In [1]:
import datetime

%matplotlib inline
import matplotlib.pyplot as pd
from IPython.display import display, YouTubeVideo

import arcgis
from arcgis.gis import GIS
from arcgis.features.analyze_patterns import interpolate_points
from arcgis.geocoding import geocode
from arcgis.features.find_locations import trace_downstream
from arcgis.features.use_proximity import create_buffers

gis = GIS(profile = "your_online_profile")

ModuleNotFoundError: No module named 'arcgis'

In [ ]:

YouTubeVideo('x4dNIfx6HVs')

In [ ]:
map = gis.map("Chennai")
map

In [ ]:
chennaipop = gis.content.search("Chennai_Population", 
                                item_type="Feature Layer", 
                                outside_org=True)[0]
chennaipop

In [ ]:
map.add_layer(chennaipop, {"renderer":"ClassedColorRenderer", 
                           "field_name": "TOTPOP_CY", 
                           "opacity":0.7})

In [ ]:
search_rainfall = gis.content.search("Chennai_precipitation", 
                              item_type="Feature Layer", 
                              outside_org=True)
if len(search_rainfall) >= 1:
    rainfall = search_rainfall[0]
else:
    # if the "Chennai_precipitation" web layer does not exist
    print("Web Layer does not exist. Re-publishing...")
    # import any pandas data frame, with an address field, as a layer in our GIS
    import pandas as pds
    df = pds.read_csv('data/Chennai_precipitation.csv')
    # Create an arcgis.features.FeatureCollection object by importing the pandas dataframe with an address field
    rainfall = gis.content.import_data(df, {"Address" : "LOCATION"})

In [ ]:
map2 = gis.map("Tamil Nadu, India")
map2

In [ ]:
map2.add_layer(rainfall, {"renderer":"ClassedSizeRenderer", 
                          "field_name":"RAINFALL" })

In [ ]:
intmap = gis.map("Tamil Nadu")
intmap

In [ ]:
intmap.add_layer(interpolated_rf['result_layer'])


In [ ]:
lakemap = gis.map("Chennai")
lakemap.height='450px'
lakemap

In [ ]:
lakemap.draw(geocode("Chembarambakkam, Tamil Nadu")[0], 
             {"title": "Chembarambakkam", "content": "Water reservoir"})
lakemap.draw(geocode("Puzhal Lake, Tamil Nadu")[0], 
             {"title": "Puzhal", "content": "Water reservoir"})
lakemap.draw(geocode("Kannampettai, Tamil Nadu")[0], 
             {"title": "Poondi Lake ", "content": "Water reservoir"})

In [ ]:
search_results = gis.content.search("Chennai_lakes", 
                                   item_type="Feature Layer",
                                   outside_org=True)
search_results

In [ ]:
chennai_lakes = search_results[2]
chennai_lakes

In [ ]:
downstream = trace_downstream(chennai_lakes)
downstream.query()

In [ ]:
floodprone_buffer = create_buffers(downstream, [ 1 ], units='Miles')

In [ ]:
lakemap.add_layer(floodprone_buffer)

In [ ]:
def exact_search(my_gis, title, owner_value, item_type_value, max_items_value=20):
    final_match = None
    search_result = my_gis.content.search(query= title + ' AND owner:' + owner_value, 
                                          item_type=item_type_value, max_items=max_items_value, outside_org=True)
    
    if "Imagery Layer" in item_type_value:
        item_type_value = item_type_value.replace("Imagery Layer", "Image Service")
    elif "Layer" in item_type_value:
        item_type_value = item_type_value.replace("Layer", "Service")
    
    for result in search_result:
        if result.title == title:
            final_match = result
            break
    return final_match

ls_water = exact_search(gis, 'Landsat GLS Multispectral', 'esri', 'Imagery Layer')
ls_water

In [ ]:
ls_water_lyr = ls_water.layers[0]

In [ ]:
from arcgis.geocoding import geocode
area = geocode("Tamil Nadu, India", out_sr=ls_water_lyr.properties.extent.spatialReference)[0]
ls_water_lyr.extent = area['extent']

In [ ]:
# data source option 
from arcgis.raster.functions import stretch, extract_band
target_img_layer = stretch(extract_band(ls_water_lyr, [5,4,3]),
                           stretch_type="percentclip", gamma=[1,1,1], dra=True)

In [ ]:
import pandas as pd
from arcgis import geometry
import datetime as dt

def filter_images(my_map, start_year, end_year):
    selected = target_img_layer.filter_by(where="(Category = 1) AND (CloudCover <=0.2)",
                                          time=[dt.datetime(start_year, 1, 1), dt.datetime(end_year, 1, 1)],
                                          geometry=arcgis.geometry.filters.intersects(ls_water_lyr.extent))
    my_map.add_layer(selected)
    
    fs = selected.query(out_fields="AcquisitionDate, GroupName, Month, DayOfYear, WRS_Row, WRS_Path")
    tdf = fs.sdf  
    return tdf

In [ ]:
satmap1 = gis.map("Pallikaranai, Tamil Nadu, India", 13)
df = filter_images(satmap1, 1991, 1992) 
df.head()

In [ ]:
satmap2 = gis.map("Pallikaranai, Tamil Nadu, India", 13)
df = filter_images(satmap2, 2009, 2010)
df.head()

In [ ]:
from ipywidgets import *

satmap1.layout=Layout(flex='1 1', padding='10px', height='300px')
satmap2.layout=Layout(flex='1 1', padding='10px', height='300px')

box = HBox([satmap1, satmap2])
box

In [ ]:
relief_centers = gis.content.search("Chennai Relief Centers")[0]

In [ ]:
reliefmap = gis.map("Chennai")
reliefmap

In [ ]:
reliefmap.add_layer(chennaipop, {"opacity":0.5})


In [ ]:
reliefmap.add_layer(relief_centers)


In [ ]:
relief_data = relief_centers.layers[0].query().sdf
relief_data.head()

In [ ]:
relief_data['No_of_pers'].sum()

In [ ]:
relief_data['No_of_pers'].describe()

In [ ]:
relief_data['No_of_pers'].hist()

In [ ]:
relief_data.shape

In [ ]:
chennai_pop_featurelayer = chennaipop.layers[0]

In [ ]:
res = arcgis.features.summarize_data.aggregate_points(  relief_centers, 
                                                        chennai_pop_featurelayer, 
                                                        False, 
                                                        ["No_of_pers Sum"])

In [ ]:
aggr_lyr = res['aggregated_layer']

In [ ]:

reliefmap.add_layer(aggr_lyr, { "renderer": "ClassedSizeRenderer", 
                               "field_name":"SUM_No_of_pers"})

In [ ]:
df = aggr_lyr.query().sdf
df.head()

In [ ]:
df = aggr_lyr.query().sdf

df2 = df[['NAME', 'SUM_No_of_pers']]
df2.set_index('NAME', inplace=True)
df2

In [ ]:
df2.plot(kind='bar')

In [ ]:
routemap = gis.map("Chennai")
routemap

In [ ]:
nehru_stadium = geocode('Jawaharlal Nehru Stadium, Chennai')[0]
routemap.draw(nehru_stadium, {"title": "Nehru Stadium", 
                              "content": "Chennai Flood Relief Center"})

In [ ]:
start_time = datetime.datetime(2015, 12, 13, 9, 0)

In [ ]:
routes = arcgis.features.use_proximity.plan_routes(
    relief_centers, 
    15, 
    15, 
    start_time, 
    nehru_stadium, 
    stop_service_time=30)
routemap.add_layer(routes['routes_layer'])

In [ ]:
routemap.add_layer(routes['assigned_stops_layer'])